In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer

In [2]:
base_url = "https://www.airlinequality.com/airport-reviews/london-heathrow-airport/"
pages = 5
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews


In [3]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,Not Verified | Best avoid this airport if you ...
1,Not Verified | Have to rescreen even if you ...
2,Not Verified | First time leaving from T2 to...
3,✅ Trip Verified | I flew from Hong Kong to He...
4,✅ Trip Verified | For the first time in 12 ye...


In [4]:
charac = '|'
df['reviews'] = df['reviews'].str.split(charac).str[1]

In [5]:
base_url = "https://www.airlinequality.com/airport-reviews/london-heathrow-airport/"

In [6]:
#creating a function for scraping data from the website
def getting_data(source):
    
    pages = 5
    page_size = 100

    reviews = []

    # for i in range(1, pages + 1):
    for i in range(1, pages + 1):


    # Create URL to collect links from paginated data
        url = f"{source}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
        response = requests.get(url)

    # Parse content
        content = response.content
        parsed_content = BeautifulSoup(content, 'html.parser')
        for para in parsed_content.find_all("div", {"class": "text_content"}):
            reviews.append(para.get_text())
            
    df = pd.DataFrame()
    df['reviews']= reviews
    df =df.dropna()
    return df
    

In [7]:
#creating a pipeline that predicts the sentiments score of user reviews
def new_pipeline():
    """
    class Preprocessing(BaseEstimator,TransformerMixin):
        def __init__(self):
            self.stopwords = set(stopwords.words('english'))
            self.lemmatizer = WordNetLemmatizer()
    
        def fit(self, X):
            return self
    
        def transform(self, X,y=None):
            return X.apply(self.preprocessing)
    
        def preprocessing(self, data):
            words = nltk.word_tokenize(data)
            words = [self.lemmatizer.lemmatize(word.lower()) for word in words if word.lower() and word.isalnum() not in self.stopwords]
            return ''.join(words)
    """
    class VaderAnalyzer(BaseEstimator, TransformerMixin):
        def __init__(self):
            self.vanalyzer = SentimentIntensityAnalyzer()
        
        def fit(self, X):
            return self
    
        def transform(self,X,y=None):
            return X.apply(self.VAnalyzer)
    
        def VAnalyzer(self, data):
            scores = self.vanalyzer.polarity_scores(data)
            return scores['compound']
    
    pipeline = Pipeline([
        ('VAnalyzer', VaderAnalyzer()),
])
    return pipeline


In [9]:
#creating a function that explores the sentiments of user reviews
def analyser(reviews):
    pipeline = new_pipeline()
    transformedd = pipeline.fit_transform(reviews['reviews'])
    sentiment=[]
    for i in transformedd:
        if i>0.0:
            sentiment.append('Positive')
        elif i<0.0:
            sentiment.append('Negative')
        else:
            sentiment.append('Neutral')
    return  transformedd, sentiment

In [10]:
def new_df(reviews):
    reviews['scores'], reviews['sentiment'] = analyser(reviews)
    return reviews

In [11]:
#creating a script that utilizes the created functions in a reusable manner for production
def main():
    df = getting_data(base_url)
    
    new = new_df(df)
    return new
    
if __name__ == "__main__":
    output = main()
    print(output)
    

                                               reviews  scores sentiment
0    Not Verified | Best avoid this airport if you ... -0.5690  Negative
1    Not Verified |   Have to rescreen even if you ...  0.0000   Neutral
2    Not Verified |   First time leaving from T2 to...  0.1501  Positive
3    ✅ Trip Verified |  I flew from Hong Kong to He...  0.5290  Positive
4    ✅ Trip Verified |  For the first time in 12 ye... -0.0516  Negative
..                                                 ...     ...       ...
495  I travel threough Heathrow on a weekly basis, ...  0.6486  Positive
496  Probably the worst airport I've seen. 1) custo... -0.7906  Negative
497  Travelled through Terminal 3 to and from the U...  0.9629  Positive
498  Heathrow airport is big but organized, great a...  0.8709  Positive
499  I was motivated to write this review after the...  0.9567  Positive

[500 rows x 3 columns]


In [ ]:
#df.to_csv("Airport reviews.csv")